In [2]:
import torch
import torch.nn as nn
import torch as pt
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch.optim as optim
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from tqdm import trange

In [3]:
# check device
device = torch.device(
    "cuda:{}".format(0) if torch.cuda.is_available() else "cpu"
)

# HCP Data using Simeon preproc

## Function definitions

In [4]:
# From DBGDGM code

def create_fc_matrices(scan, window_size=30, step_size=10):
    """
    Create functional connectivity matrices using a sliding window approach.
    Function from DBGDGM code.
    """
    n_timepoints = scan.shape[1]
    fc_matrices = []

    for start in range(0, n_timepoints - window_size + 1, step_size):
        window = scan[:, start:start + window_size]
        correlation_matrix = np.corrcoef(window)   # pearson correlation 
        fc_matrices.append(correlation_matrix)

    return fc_matrices


def threshold_fc_matrix(fc_matrix, percentile=5):
    """
    Threshold the FC matrix to keep only the top percentile connections.
    """
    threshold = np.percentile(fc_matrix[np.tril_indices_from(fc_matrix, k=-1)], 100 - percentile)   
    graph = (fc_matrix > threshold).astype(int)
    np.fill_diagonal(graph, 0)  # remove self-edges
    return graph


def create_networkx_graph(matrix):
    G = nx.Graph(matrix)
    return G


def convert_to_pyg_graph(nx_graph, label):
    graph = nx.Graph(nx_graph)
    edges = torch.tensor(list(graph.edges), dtype=torch.long).t().contiguous()
    x = torch.tensor(np.identity(graph.number_of_nodes()), dtype=torch.float)
    y = torch.tensor([label], dtype=torch.long)
    return Data(x=x, edge_index=edges, y=y)

In [5]:
# From DBGDSL code

def create_fc_matrices_dbgsl(scan, window_size=30, step_size=10):
    """
    Create functional connectivity matrices using a sliding window approach.
    Function created using DBGSL code to split data into windows.
    """
    n_timepoints = scan.shape[1]
    t_repetition = (n_timepoints - 2*(window_size - 1) - 1)//(step_size + 1)
    fc_matrices = []

    for t in range(t_repetition):
        window = scan[:, t*step_size:t*step_size+window_size]
        correlation_matrix = (np.corrcoef(window) + 1) / 2   # normalized pearson correlation 
        fc_matrices.append(correlation_matrix)

    return fc_matrices

def get_x_split(x: pt.Tensor, stride, len_window, t_repetition) -> pt.Tensor:
    x_split = pt.stack([x[:, :, t*stride:t*stride+len_window] for t in range(t_repetition)], 1)
    return x_split.float()

def get_node_features(x_split: pt.Tensor) -> pt.Tensor:
    x_split_avg = pt.mean(x_split, -1, keepdim=True)
    x_split_std = pt.std(x_split, -1, keepdim=True)
    x_split_cov = pt.matmul(x_split - x_split_avg, pt.transpose(x_split - x_split_avg, 2, 3))
    node_features = x_split_cov/pt.matmul(x_split_std, pt.transpose(x_split_std, 2, 3))
    return node_features

def get_coo(adjacency_matrix: pt.Tensor) -> pt.Tensor:
    i = 0
    edge_indices = pt.nonzero(adjacency_matrix > 0, as_tuple=False).T
    edge_index_batch = pt.clone(edge_indices[1:3, :])
    for t in range(len(edge_indices[0])):
        if i < edge_indices[0][t]:
            i = i + 1
            n_nodes = max(edge_indices[1][t-1], edge_indices[2][t-1])+1
            edge_index_batch[0][t:] = edge_index_batch[0][t:] + n_nodes
            edge_index_batch[1][t:] = edge_index_batch[1][t:] + n_nodes
    edge_attr_batch = adjacency_matrix[adjacency_matrix > 0].unsqueeze(-1)
    batch = edge_indices[0]
    return edge_index_batch, edge_attr_batch, batch


def construct_graph(x_ebd: torch.Tensor) -> torch.Tensor:
    batch_size = x_ebd.shape[0]
    batch_adjacency_matrices = []

    for i in range(batch_size):
        sample = x_ebd[i]
        sample = F.softmax(sample, -1)
        adjacency_matrix = torch.matmul(sample, sample.transpose(1, 2))
        batch_adjacency_matrices.append(adjacency_matrix)
    adjacency_matrix_batch = torch.stack(batch_adjacency_matrices, dim=0)

    return adjacency_matrix_batch

def sparsify(adjacency_matrix_batch: torch.Tensor) -> torch.Tensor:
    threshold = torch.nn.parameter.Parameter(torch.full((1,), -5.0))
    batch_sparse_adjacency = []

    for i in range(adjacency_matrix_batch.size(0)):
        adjacency_matrix = adjacency_matrix_batch[i]
        sparse_adjacency = F.relu(adjacency_matrix - torch.sigmoid(threshold))
        batch_sparse_adjacency.append(sparse_adjacency)
    sparse_adjacency_batch = torch.stack(batch_sparse_adjacency, dim=0)

    return sparse_adjacency_batch

In [6]:
# From DBGSL code

class InceptionTC(nn.Module):
    def __init__(self, itcn_d, dilation):
        super(InceptionTC, self).__init__()
        self.t_conv_0 = nn.Conv1d(itcn_d, itcn_d//3, kernel_list[0], dilation=dilation, padding=(kernel_list[0]-1)*dilation)
        self.t_conv_1 = nn.Conv1d(itcn_d, itcn_d//3, kernel_list[1], dilation=dilation, padding=(kernel_list[1]-1)*dilation)
        self.t_conv_2 = nn.Conv1d(itcn_d, itcn_d//3, kernel_list[2], dilation=dilation, padding=(kernel_list[2]-1)*dilation)
        self.bn = nn.BatchNorm1d(itcn_d)

    def forward(self, x_split, dilation):
        x_cat = [
            self.clip_end(self.t_conv_0(x_split), 0, dilation),
            self.clip_end(self.t_conv_1(x_split), 1, dilation),
            self.clip_end(self.t_conv_2(x_split), 2, dilation)
        ]
        x_cat = torch.cat(x_cat, 1)
        x_out = F.relu(self.bn(x_cat))
        return x_out

    def clip_end(self, x, i, dilation):
        padding = (kernel_list[i] - 1) * dilation
        x = x[:, :, :-padding].contiguous()
        return x


class ITCN(nn.Module):
    def __init__(self, batch_size, n_neurons, itcn_d, t_repetition, kernel_list):
        super(ITCN, self).__init__()
        self.batch_size = batch_size
        self.n_neurons = n_neurons
        self.itcn_d = itcn_d
        self.t_repetition = t_repetition
        self.kernel_list = kernel_list
        self.inception_tc_2 = InceptionTC(itcn_d, 2)  
        self.inception_tc_4 = InceptionTC(itcn_d, 4)
        self.inception_tc_6 = InceptionTC(itcn_d, 6)

    def forward(self, x_split):
        x_split = x_split.reshape(self.batch_size * self.n_neurons, self.itcn_d, self.t_repetition)
        x_split = self.inception_tc_2(x_split, 2)
        x_split = self.inception_tc_4(x_split, 4)
        x_split = self.inception_tc_6(x_split, 6)
        x_split = x_split.reshape(self.batch_size, self.t_repetition, self.n_neurons, self.itcn_d)
        return x_split


class RegionEmbedding(nn.Module):
    def __init__(self, len_window, itcn_d, ebd_d, batch_size, n_neurons, t_repetition, kernel_list):
        super(RegionEmbedding, self).__init__()
        self.input_layer = nn.Linear(len_window, itcn_d)
        self.itcn_layer = ITCN(batch_size, n_neurons, itcn_d, t_repetition, kernel_list)
        self.output_fc = nn.Sequential(nn.Linear(itcn_d, itcn_d), nn.ReLU(), nn.Linear(itcn_d, ebd_d))

    def forward(self, x_split):
        x_split = self.input_layer(x_split)
        x_split = self.itcn_layer(x_split)
        x_split = self.output_fc(x_split)
        return x_split


class SpatialAttention(nn.Module):
    def __init__(self, n_neurons, tau):
        super(SpatialAttention, self).__init__()
        n_neurons_ebd = int(tau * n_neurons)
        self.spatial_attn = nn.Sequential(
            nn.Linear(n_neurons, n_neurons_ebd, bias=False),
            nn.ReLU(),
            nn.Linear(n_neurons_ebd, n_neurons, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x_ebd):
        x_spatial_attn = torch.mean(x_ebd, -1)
        x_spatial_attn = self.spatial_attn(x_spatial_attn)
        x_spatial_attn = x_spatial_attn.unsqueeze(-1)
        return x_spatial_attn


class TemporalAttention(nn.Module):
    def __init__(self, t_repetition, tau, n_neurons, ebd_d):
        super(TemporalAttention, self).__init__()
        T_ebd = int(tau * t_repetition)
        self.temporal_attn = nn.Sequential(
            nn.Linear(t_repetition, T_ebd, bias=False),
            nn.ReLU(),
            nn.Linear(T_ebd, t_repetition, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x_ebd):
        x_temporal_attn = x_ebd.view(-1, t_repetition, n_neurons * ebd_d)
        x_temporal_attn = torch.mean(x_temporal_attn, -1)
        x_temporal_attn = self.temporal_attn(x_temporal_attn)
        x_temporal_attn = x_temporal_attn.view(-1, t_repetition, 1, 1)
        return x_temporal_attn


class MyModel(nn.Module):
    def __init__(self, len_window, itcn_d, ebd_d, batch_size, n_neurons, t_repetition, tau, kernel_list):
        super(MyModel, self).__init__()
        self.region_embd = RegionEmbedding(len_window, itcn_d, ebd_d, batch_size, n_neurons, t_repetition, kernel_list)
        self.spat_attention = SpatialAttention(n_neurons, tau)
        self.temp_attention = TemporalAttention(t_repetition, tau, n_neurons, ebd_d)

    def forward(self, x):
        x_ebd = self.region_embd(x)
        x_spatial_attention = self.spat_attention(x_ebd)
        x_ebd = x_spatial_attention * x_ebd
        x_temporal_attn = self.temp_attention(x_ebd)
        x_ebd = x_temporal_attn * x_ebd
        return x_ebd


In [7]:
def cos_similar(p, q):
    sim_matrix = p.matmul(q.transpose(-2, -1))
    a = torch.norm(p, p=2, dim=-1)
    b = torch.norm(q, p=2, dim=-1)
    sim_matrix /= a.unsqueeze(-1)
    sim_matrix /= b.unsqueeze(-2)
    sim_matrix = torch.where(torch.isnan(sim_matrix), torch.full_like(sim_matrix, 0), sim_matrix)
    return sim_matrix

## Masking Generator

In [8]:
class TubeMaskingGenerator:
    def __init__(self, input_size, mask_ratio):
        self.frames, self.num_patches_per_frame = input_size
        self.total_patches = self.frames * self.num_patches_per_frame 
        self.num_masks_per_frame = int(mask_ratio * self.num_patches_per_frame)
        self.total_masks = self.frames * self.num_masks_per_frame

    def __repr__(self):
        repr_str = "Masks: total patches {}, mask patches {}".format(
            self.total_patches, self.total_masks
        )
        return repr_str

    def __call__(self):
        mask_per_frame = np.hstack([
            np.zeros(self.num_patches_per_frame - self.num_masks_per_frame),
            np.ones(self.num_masks_per_frame),
        ])
        mask = np.tile(mask_per_frame, (self.frames, 1)).flatten()
        return mask 

## Example of tensor dimensions

In [9]:
# ----------- One raw as example -------------------------

data_path = '../data/hcp/raw'
file_name = '100206_0.npy'
file_path = os.path.join(data_path, file_name)
print('FILE_PATH: ', file_path)
time_series_data = np.load(file_path)[:, :490]
print('time_series_data SHAPE: ', time_series_data.shape)
label = int(os.path.basename(file_path).split('_')[-1].split('.')[0])

fc_matrices = create_fc_matrices(time_series_data) # --> dbgdgm way of slicing the timeseries
fc_matrices_tn = torch.tensor(fc_matrices).reshape(1, len(fc_matrices), 360, 360)
print('fc_matrices_tn SHAPE (slicing as in DBGDGM code): ', fc_matrices_tn.shape)
fc_matrices_dbgsl = create_fc_matrices_dbgsl(time_series_data)
fc_matrices_dbgsl_tn = torch.tensor(fc_matrices_dbgsl).reshape(1, len(fc_matrices_dbgsl), 360, 360) # --> dbgsl way of slicing the timeseries
print('fc_matrices_39 SHAPE (slicing as in DBGSL code): ', fc_matrices_dbgsl_tn.shape)

graphs = [threshold_fc_matrix(fc) for fc in fc_matrices] 
graph = graphs[0]
graph_nx = convert_to_pyg_graph(graph, label)

# ----------- One raw as example -------------------------


FILE_PATH:  ../data/hcp/raw/100206_0.npy
time_series_data SHAPE:  (360, 490)


/tmp/ipykernel_2733097/983908113.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  fc_matrices_tn = torch.tensor(fc_matrices).reshape(1, len(fc_matrices), 360, 360)


fc_matrices_tn SHAPE (slicing as in DBGDGM code):  torch.Size([1, 47, 360, 360])
fc_matrices_39 SHAPE (slicing as in DBGSL code):  torch.Size([1, 39, 360, 360])


In [10]:
# Define input size and masking ratio
input_size = (490, 360)  # (time points, brain regions)
mask_ratio = 0.50

# Create a TubeMaskingGenerator instance
masking_generator = TubeMaskingGenerator(input_size, mask_ratio)

# Generate the tube mask
tube_mask = masking_generator()
shuffled = np.random.permutation(tube_mask.reshape(time_series_data.T.shape).T)

# Apply the tube mask to the data
masked_data = time_series_data * shuffled
masked_data = masked_data

### Hyperparameters

In [11]:
batch_size = 1   # remember to change
T = 490
len_window = 30   
stride = 10  
t_repetition = (T - 2*(len_window - 1) - 1)//(stride + 1)
print('t_repetition: ', t_repetition)

# Training hyperparameters
device = 'cpu'
n_neurons = 360
lr = 0.003  # learning rate, 0.003
n_itcn_layers = 3
n_gru_layers = 1
tau = 0.1
kernel_list = [3, 5, 7]
itcn_d = 9
ebd_d = 3
gcn_d = 5
n_classes = 1

time_series_torch = torch.from_numpy(time_series_data).reshape(batch_size, time_series_data.shape[0], time_series_data.shape[1])
print('RESHAPING time_series_data: ', time_series_torch.shape)

time_series_masked = torch.from_numpy(masked_data).reshape(batch_size, masked_data.shape[0], masked_data.shape[1])
print('RESHAPING masked_data: ', time_series_masked.shape)

x_split = get_x_split(time_series_masked, stride, len_window, t_repetition)
print('SPLITTING WINDOW time_series_data: ', x_split.shape)

t_repetition:  39
RESHAPING time_series_data:  torch.Size([1, 360, 490])
RESHAPING masked_data:  torch.Size([1, 360, 490])
SPLITTING WINDOW time_series_data:  torch.Size([1, 39, 360, 30])


In [12]:
# Instantiate your model
model = MyModel(len_window, itcn_d, ebd_d, batch_size, n_neurons, t_repetition, tau, kernel_list)

## Training

#### MSE minimization

In [26]:
subject = 0
n_episodes = 1000   # n_epochs, remember to change
save_interval = 50

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_arr = np.zeros(n_episodes)

saved = []
ground_truth = dataset_ground[subject].float()
epochs = trange(n_episodes)
for epoch in epochs:
    optimizer.zero_grad()
    outputs = model(dataset_input[subject])
    adjacency_matrix = construct_graph(outputs)
    # sparse_adjacency = sparsify(adjacency_matrix)
    loss = criterion(adjacency_matrix, ground_truth)
    loss.backward()
    optimizer.step()
    loss_arr[epoch] = loss.item()
    epochs.set_postfix({'Loss': loss.item()})

    if epoch % save_interval == 0:
        saved.append(adjacency_matrix)
    

100%|██████████| 1000/1000 [01:33<00:00, 10.70it/s, Loss=0.0137]
